### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [1]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import ab_power_trader
import json
import http.client
import certifi
import ssl
import os

In [27]:
# Pull current day of data (current_df)
from datetime import datetime, timedelta
import pandas as pd
import pull_nrg_data
from google.oauth2 import service_account
from pandasql import sqldf

def get_streams():
    streams = pd.read_csv('stream_codes.csv')
    lst = [int(id) for id in streams[(streams['timeInterval']=='1 hr') & (streams['intervalType']=='supply')]['streamId']]
    return lst
[86, 322684, 322677, 87, 85, 23695, 322665, 23694]
# Path to Google auth credentials
#credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])

if __name__ == '__main__':
    streamIds = [86, 322684, 322677, 87, 85, 23695, 322665, 23694]
    current_df = pd.DataFrame([])
    today = datetime.now()
    for id in streamIds:
        accessToken, tokenExpiry = pull_nrg_data.getToken()
        try:
            APIdata = pull_nrg_data.pull_data(today.strftime('%m/%d/%Y'), today.strftime('%m/%d/%Y'), id, accessToken, tokenExpiry)
            pull_nrg_data.release_token(accessToken)
            APIdata['timeStamp'] = pd.to_datetime(APIdata['timeStamp'])
            current_df = pd.concat([current_df, APIdata], axis=0)
        except:
            pull_nrg_data.release_token(accessToken)
            pass
    query = '''
        SELECT  
                fuelType,
                strftime('%Y', timeStamp) AS year,
                strftime('%m', timeStamp) AS month,
                strftime('%d', timeStamp) AS day,
                strftime('%H', timeStamp) AS hour,
                avg(value) 
        FROM current_df
        GROUP BY fuelType, year, month, day, hour
        '''
sqldf(query, globals())

,fuelType,year,month,day,hour,avg(value)
0,Biomass & Other,2022,07,27,00,235.916667
1,Biomass & Other,2022,07,27,01,234.416667
2,Biomass & Other,2022,07,27,02,234.333333
3,Biomass & Other,2022,07,27,03,232.750000
4,Biomass & Other,2022,07,27,04,232.833333
...,...,...,...,...,...,...
131,Wind,2022,07,27,12,62.916667
132,Wind,2022,07,27,13,55.750000
133,Wind,2022,07,27,14,49.916667
134,Wind,2022,07,27,15,48.500000


,fuelType,year,month,day,hour,avg(value)
0,Biomass & Other,2022,07,27,00,235.916667
1,Biomass & Other,2022,07,27,01,234.416667
2,Biomass & Other,2022,07,27,02,234.333333
3,Biomass & Other,2022,07,27,03,232.750000
4,Biomass & Other,2022,07,27,04,232.833333
...,...,...,...,...,...,...
115,Wind,2022,07,27,10,30.416667
116,Wind,2022,07,27,11,66.833333
117,Wind,2022,07,27,12,62.916667
118,Wind,2022,07,27,13,55.750000


In [36]:
pull_nrg_data.release_token(accessToken)

In [42]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = '6Bkess-sojuUO1hQp3v8yw95UtGAWTf1glDZ9HiIhGOdAOdwFxqJDapWeeq1roBpYE_6jMgbYENN5qe4nyaTjX5YsTETWBpPwRrNUl24ClVdJ3WYiDVnRfF_JuDkASPHluCNv3qvSuj4BNbZvsEzToR6AGxbqGPgySAy2uDPNb19HIMgGy7dVo53hH_5An33C3ockq5plH_k9Zs2FnED94EDr9DJ4OVFe5qT60DgIMkK3DsHrnEZmvmvhH2Igu3rpMp8myfgLi7Z3xeAVWEv1VlNAX9x9LuK7kHOdjekf_ZBWkW0clkFP4WnyjO420GzEmbDeY6ByerjsacekazqaoQgjzA'
release_token(accessToken)

token released


In [22]:
# Pull data from BQ
import os
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# Path to Google auth credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ryan-bulger/power-trader/google-big-query.json'
sql = "SELECT * FROM nrgdata.grouped_data"
df = bigquery.Client().query(sql).to_dataframe()

In [61]:
def stream_data(streamIds, streamNames, years):
    stream_df = pd.DataFrame([])
    for id in streamIds:
        server = 'api.nrgstream.com'
        year_df = pd.DataFrame([])
        for yr in years:
            accessToken, tokenExpiry = pull_nrg_data.getToken()
            # Define start & end dates
            startDate = date(yr,1,1).strftime('%m/%d/%Y')
            endDate = date(yr,12,31).strftime('%m/%d/%Y')
            # NRG API connection
            path = f'/api/StreamData/{id}?fromDate={startDate}&toDate={endDate}'
            headers = {'Accept': 'Application/json', 'Authorization': f'Bearer {accessToken}'}
            context = ssl.create_default_context(cafile=certifi.where())
            conn = http.client.HTTPSConnection(server, context=context)
            conn.request('GET', path, None, headers)
            res = conn.getresponse()
            # Load json data from API & create pandas df
            jsonData = json.loads(res.read().decode('utf-8'))
            df = pd.json_normalize(jsonData, record_path='data')
            # Close NRG API connection
            conn.close()
            # Concat years for each stream
            year_df = pd.concat([year_df,df], axis=0)
            # Release NRG API access token
            pull_nrg_data.release_token(accessToken)
        # Rename year_df cols
        year_df.rename(columns={0:'timeStamp', 1:f'{streamNames[id]}'}, inplace=True)
        print(year_df)
        # Change timeStamp to datetime
        year_df['timeStamp'] = pd.to_datetime(year_df['timeStamp'])
        # Re-index the year_df
        year_df.set_index('timeStamp', inplace=True)
        # Join year_df to outages dataframe
        stream_df = pd.concat([stream_df,year_df], axis=1, join='outer')
    return stream_df

streamIds = [44648, 118361, 322689, 118362, 147262, 322675, 322682, 44651]
#streamIds = [44648]
streamNames = {44648:'Coal', 118361:'Gas', 322689:'Dual Fuel', 118362:'Hydro', 147262:'Wind', 322675:'Solar', 322682:'Energy Storage', 44651:'Biomass & Other'}
years = [datetime.now().year, datetime.now().year+1, datetime.now().year+2]
outage_df = stream_data(streamIds, streamNames, years)
#print(outage_df)
# Reset index so dataframe can be plotted with Altair
outage_df.reset_index(inplace=True)
outage_df = pd.melt(outage_df, 
                id_vars=['timeStamp'],
                value_vars=['Coal', 'Gas', 'Dual Fuel', 'Hydro', 'Wind', 'Solar', 'Energy Storage', 'Biomass & Other'],
                var_name='Source',
                value_name='Value')
outage_df

            timeStamp Coal
0   Jan 01 2022 00:00  700
1   Feb 01 2022 00:00    0
2   Mar 01 2022 00:00   80
3   Apr 01 2022 00:00  310
4   May 01 2022 00:00    0
5   Jun 01 2022 00:00    0
6   Jul 01 2022 00:00    0
7   Aug 01 2022 00:00    0
8   Sep 01 2022 00:00    0
9   Oct 01 2022 00:00  470
10  Nov 01 2022 00:00   30
11  Dec 01 2022 00:00    0
0   Jan 01 2023 00:00    0
1   Feb 01 2023 00:00    0
2   Mar 01 2023 00:00    0
3   Apr 01 2023 00:00    0
4   May 01 2023 00:00    0
5   Jun 01 2023 00:00    0
6   Jul 01 2023 00:00    0
7   Aug 01 2023 00:00    0
8   Sep 01 2023 00:00    0
9   Oct 01 2023 00:00    0
10  Nov 01 2023 00:00    0
11  Dec 01 2023 00:00    0
0   Jan 01 2024 00:00    0
1   Feb 01 2024 00:00    0
2   Mar 01 2024 00:00    0
3   Apr 01 2024 00:00    0
4   May 01 2024 00:00    0
5   Jun 01 2024 00:00    0
6   Jul 01 2024 00:00    0
            timeStamp   Gas
0   Jan 01 2022 00:00   740
1   Feb 01 2022 00:00   820
2   Mar 01 2022 00:00   990
3   Apr 01 2022 00:00  1

,timeStamp,Source,Value
0,2022-01-01,Coal,700
1,2022-02-01,Coal,0
2,2022-03-01,Coal,80
3,2022-04-01,Coal,310
4,2022-05-01,Coal,0
...,...,...,...
243,2024-03-01,Biomass & Other,30
244,2024-04-01,Biomass & Other,30
245,2024-05-01,Biomass & Other,30
246,2024-06-01,Biomass & Other,30


In [62]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
df

,hourly,daily
2016-01-01 00:00:00,0,0.0
2016-01-01 01:00:00,1,0.0
2016-01-01 02:00:00,2,0.0
2016-01-01 03:00:00,3,0.0
2016-01-01 04:00:00,4,0.0
...,...,...
2016-01-06 20:00:00,140,5.0
2016-01-06 21:00:00,141,5.0
2016-01-06 22:00:00,142,5.0
2016-01-06 23:00:00,143,5.0
